In [1]:
import sys
import os

#library_path is the path where the Optimpy library is located.
library_path = "/home/dell/Documentos/Git_proejcts/pyristic/"
#library_path = "/Users/adrianamenchacamendez/Documentos/enes_morelia/papime/optimizacion-con-metaheuristicas/"
sys.path.append(os.path.abspath(library_path))

In [2]:
from pyristic.heuristic.Tabu_search import TabuSearch as TS
from pyristic.heuristic.SimulatedAnnealing_search import SimulatedAnnealing as SA
from pyristic.heuristic.GeneticAlgorithm_search import Genetic as GA
from pyristic.utils.helpers import *
from pyristic.utils.operators import selection,mutation,crossover

In [3]:
from pprint import pprint
import numpy as np 
import random 
import copy 

In [4]:
np.random.seed(10)

## Problema de la mochila

In [5]:
class knapsackFunction:
    def __init__(self, benefits: (list, np.ndarray)):
        self.benefits = np.array(benefits)
        
    def __call__(self,X: np.ndarray):
        return -1 * np.dot(X,self.benefits)

class knapsackAptitude:
    def __init__(self, benefits: (list, np.ndarray)):
        self.benefits = np.array(benefits)
        
    def __call__(self,X:np.ndarray):
        return np.dot(X,self.benefits) 

           
class knapsackConstraint:
    
    def __init__(self, weights: (list,np.ndarray), cappacity):
        self.weights = np.array(weights)
        self.cappacity = cappacity
        
    def __call__(self,X: np.ndarray):
        result  = np.dot(X, self.weights)
        self.__doc__="{} <= {}".format(result,self.cappacity)
        return result <= self.cappacity
    


In [6]:
n0 = 5
p0 = [2,4,3,5,1]
w0 = [1,2,3,1,2]
c0 = 5
f0 = knapsackFunction(p0)
g0 = knapsackConstraint(w0,c0)


#### Instancia 1

In [7]:
n1 = 50
p1 = [60, 52, 90, 57, 45, 64, 60, 45, 63, 94, 44, 90, 66, 64, 32, 39, 91, 40, 73, 61, 82, 94, 39, 68, 94, 98, 80, 79, 73, 99, 49, 56, 69, 49, 82, 99, 65, 34, 31, 85, 67, 62, 56, 38, 54, 81, 98, 63, 48, 83]
w1 = [38, 20, 21, 21, 37, 28, 32, 30, 33, 35, 29, 32, 35, 24, 28, 29, 22, 34, 31, 36, 36, 28, 38, 25, 38, 37, 20, 23, 39, 31, 27, 20, 38, 38, 36, 28, 39, 22, 23, 22, 21, 24, 23, 33, 31, 30, 32, 30, 22, 37]
c1 = 500

In [8]:
f1 = knapsackFunction(p1)
g1 = knapsackConstraint(w1,c1)

#### Instancia 2

In [9]:
n2 = 100
p2 =  np.random.randint(10,100,n2)
w2 =  np.random.randint(15,80,n2)
c2 =  1000

In [10]:
f2 = knapsackFunction(p2)
g2 = knapsackConstraint(w2,c2)


#### Instancia 3

In [11]:
n3 = 150
p3 =  np.random.randint(10,100,n3)
w3 =  np.random.randint(15,80,n3)
c3 =  1500

In [12]:
f3 = knapsackFunction(p3)
g3 = knapsackConstraint(w3,c3)

### Búsqueda tabú configuración

#### Solución inicial

In [13]:
class getInitialSolution:
    
    def __init__(self,n,p,w,c):
        self.n = n
        self.p = p
        self.w = w
        self.c = c
    
    def __call__(self):
        #Empty backpack
        x = [0 for i in range(self.n)]
        weight_x = 0

        #Random order to insert objects.
        objects = list(range(self.n))
        np.random.shuffle(objects)

        for o in  objects:
            #Check the constraint about capacity.
            if weight_x + self.w[o] <= self.c:
                x[o] = 1
                weight_x += self.w[o]
     
        return np.array(x)

#### Configuración metaheurística

In [14]:
class knapsack_TS(TS):
    
    def __init__(self, f_ : function_type , constraints_: list):
        super().__init__(f_,constraints_)
        
        
    def get_neighbors(self, x : np.ndarray,**kwargs) -> list:   
        neighbors_list = []

        for i in range(len(x)):
            x[i] ^= 1 #1
            neighbors_list+=[copy.deepcopy(x)]
            x[i] ^= 1 
            
        return neighbors_list
        
    def encode_change(self, neighbor : (list,np.ndarray), x : (list,np.ndarray),**kwargs) -> list: #2
        
        x_ = [None,None]
        
        for i in range(len(x)):
            if x[i] != neighbor[i]:
                return [i,neighbor[i]]
            
        return x_

### Búsqueda recocido simulado

#### Solución inicial

In [15]:
class initializer:

    def __init__(self, n,p,w,c):
        self.n = n
        self.p = p
        self.w = w
        self.c = c
        
        self.initGISP()
        self.getSolutionByGISP()
    
    def initGISP(self):
        self.GISP_Arr=[]
        for i in range(self.n):
            self.GISP_Arr.append((self.p[i]/self.w[i],i))

        self.GISP_Arr.sort(reverse=True)
    
    def getSolutionByGISP(self):
        X = [0 for i in range(self.n)]
        current_weight = 0
        for i in range(self.n):
            ind = self.GISP_Arr[i][1]
            if current_weight+ self.w[ind] <= self.c:
                current_weight+=self.w[ind]
                X[ind] = 1
        self.solution =  np.array(X)


#### Configuración metaheurística

In [16]:
class knapsack_SA(SA):

    def __init__(self, f_ : function_type , constraints_: list, n,p,w,c, GISP_Arr):
        super().__init__(f_,constraints_)
        self.n = n
        self.p = p
        self.w = w
        self.c = c
        self.GISP_Arr = GISP_Arr
        
    def generate_neighbor(self, x: np.ndarray) -> np.ndarray:
        x_ = x.copy()
        N = len(x_)
        ind = random.randint(0, N-1)
        x_[ind] ^= 1
        return x_
    
    def repair_neighbor(self, x: np.ndarray) -> np.ndarray:
        #Get the total weight
        total_weight=0
        for i in range(self.n):
            if x[i]:
                total_weight += self.w[i]
        
        for i in range(self.n):
            ind = self.GISP_Arr[self.n - (1+i)][1] #Lowest
            
            if x[ind]:
                total_weight -= self.w[ind]
                x[ind] = 0
            
            if total_weight <= self.c:
                break
                
    def improve_neighbor(self, x: np.ndarray) -> np.ndarray:
        
        #Get the total weight
        total_weight=0
        for i in range(self.n):
            if x[i]:
                total_weight += self.w[i]
                
        for i in range(self.n):
            ind = self.GISP_Arr[i][1]
            if total_weight + self.w[ind] > self.c:
                continue
                
            if x[ind] == 0 :
                total_weight += self.w[ind]
                x[ind] = 1
                
    def get_neighbor(self, x : np.ndarray) -> np.ndarray: 
        
        neighbor_ = self.generate_neighbor(x)
            
        if(self.is_valid(neighbor_)):
            return neighbor_
        
        #RI strategy
        self.repair_neighbor(neighbor_)
        self.improve_neighbor(neighbor_)
        
        return neighbor_

### Búsqueda Algoritmos genéticos

#### Configuración de operadores

In [17]:
configuration_knapsack = (GeneticConfig()
                             .cross(crossover.n_point_crossover(2))
                             .mutate(mutation.binary_mutator(.1))
                             .survivor_selection(selection.merge_selector())
                             .parent_selection(selection.tournament_sampler(chunks_=5,prob_=0.8))
                             )

#### Configuración de metaheurística

In [18]:
def fixer(individual,weights, cappacity):
    indices = []
    current_weight = 0
    for index,item in enumerate(individual):
        if item == 1:
            indices.append(index)
            current_weight += weights[index]

    permutation_indices = np.random.permutation(indices)
    new_solution = np.zeros(len(individual))
    new_solution[indices] = 1

    for i in permutation_indices:
        if current_weight <= cappacity:
            break
        current_weight -= weights[i]
        new_solution[i] = 0

    return new_solution     
    
class knapsack_GA(GA):
    def __init__(self,  function,\
                        decision_variables,\
                        constraints,\
                        config,\
                        n,\
                        p,\
                        w,\
                        c):
        super().__init__(function, decision_variables, constraints, [], config)
        self.n = n
        self.p = p
        self.w = w
        self.c = c
        
    def initialize_population(self, **kwargs) -> np.ndarray:
        individuals = []
        
        for i in range(self.logger['population_size']):
            individual_ = np.random.randint(2, size= self.n)
            if self.is_invalid(individual_):
                individuals += [fixer(individual_, self.w, self.c)]
            else:
                individuals += [individual_]
        return np.array(individuals)
 
    def fixer(self, ind: int) -> np.ndarray:
        return fixer(self.logger['offspring_population_x'][ind],self.w, self.c)

### Resultados 

#### Instancia 1

In [19]:
AG_instancia1  = knapsack_GA(     
                    function= knapsackAptitude(p1),\
                    decision_variables=n1,\
                    constraints= [g1],\
                    config=configuration_knapsack,\
                    n = n1,\
                    p = p1,\
                    w = w1,\
                    c = c1
                )

In [76]:
args = (30,19, 0.7, 0.7, False)
statistics = get_stats(AG_instancia1, 30, args,{},showAll=False)



In [77]:
pprint(statistics)

{'Best solution': {'f': 1407.0,
                   'x': array([0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.])},
 'Mean': 1450.0666666666666,
 'Median': 1454.5,
 'Standard deviation': 25.656621410899408,
 'Worst solution': {'f': 1490.0,
                    'x': array([0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1.])}}


In [62]:
initial_solution = initializer(n1,p1,w1,c1)
TS_instancia1 = knapsack_TS(
    f1,
    [g1]
    )

In [63]:
args = (initial_solution.solution,1140,n1//2,False)
statistics = get_stats(TS_instancia1, 30, args,showAll=False)

In [64]:
pprint(statistics)

{'Best solution': {'f': -1542,
                   'x': array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0])},
 'Mean': -1542.0,
 'Median': -1542.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': -1542,
                    'x': array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0])}}


In [65]:
SA_instancia1 = knapsack_SA(f1, [g1],n1,p1,w1,c1,initial_solution.GISP_Arr)

In [66]:
args = (initial_solution.solution,1000.0,6.862158749582714e-50)
statistics = get_stats(SA_instancia1, 30, args,showAll=False)

In [67]:
pprint(statistics)

{'Best solution': {'f': -1555,
                   'x': array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0])},
 'Mean': -1549.1666666666667,
 'Median': -1552.0,
 'Standard deviation': 5.007217013693557,
 'Worst solution': {'f': -1542,
                    'x': array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0])}}


#### Instancia 2

In [78]:
AG_instancia2  = knapsack_GA(     
                    function= knapsackAptitude(p2),\
                    decision_variables=n2,\
                    constraints= [g2],\
                    config=configuration_knapsack,\
                    n = n2,\
                    p = p2,\
                    w = w2,\
                    c = c2
                )

In [80]:
args = (30,19, 0.7, 0.7, False)
statistics = get_stats(AG_instancia2, 30, args,{},showAll=False)

In [81]:
pprint(statistics)

{'Best solution': {'f': 1837.0,
                   'x': array([0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.])},
 'Mean': 1952.0666666666666,
 'Median': 1956.5,
 'Standard deviation': 51.33610382134152,
 'Worst solution': {'f': 2060.0,
                    'x': array([0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 

In [82]:
initial_solution = initializer(n2,p2,w2,c2)
TS_instancia2 = knapsack_TS(
    f2,
    [g2]
    )

In [83]:
args = (initial_solution.solution,1140,n2//2,False)
statistics = get_stats(TS_instancia2, 30, args,showAll=False)

In [84]:
pprint(statistics)

{'Best solution': {'f': -2401,
                   'x': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])},
 'Mean': -2401.0,
 'Median': -2401.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': -2401,
                    'x': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])}}


In [85]:

SA_instancia1 = knapsack_SA(f2, [g2],n2,p2,w2,c2,initial_solution.GISP_Arr)

In [86]:
args = (initial_solution.solution,1000.0,6.862158749582714e-50)
statistics = get_stats(SA_instancia1, 30, args,showAll=False)

In [87]:
pprint(statistics)

{'Best solution': {'f': -2417,
                   'x': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])},
 'Mean': -2411.133333333333,
 'Median': -2412.0,
 'Standard deviation': 5.840852297015869,
 'Worst solution': {'f': -2401,
                    'x': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])}}


#### Instancia 3

In [88]:
AG_instancia3  = knapsack_GA(     
                    function= knapsackAptitude(p3),\
                    decision_variables=n3,\
                    constraints= [g3],\
                    config=configuration_knapsack,\
                    n = n3,\
                    p = p3,\
                    w = w3,\
                    c = c3
                )

In [89]:
args = (30,19,0.7, 0.7, False)
statistics = get_stats(AG_instancia3, 30, args,{},showAll=False)

In [90]:
pprint(statistics)

{'Best solution': {'f': 2653.0,
                   'x': array([1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.])},
 'Mean': 2861.2,
 'Median': 2860.0,
 'Standard deviation': 86.73653593882261,
 'Worst solution': {'f': 3050.0,
                    'x': array([0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 

In [91]:
initial_solution = initializer(n3,p3,w3,c3)
TS_instancia3 = knapsack_TS(
    f3,
    [g3]
    )

In [92]:
args = (initial_solution.solution,1140,n3//2,False)
statistics = get_stats(TS_instancia3, 30, args,showAll=False)

In [93]:
pprint(statistics)

{'Best solution': {'f': -3768,
                   'x': array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0])},
 'Mean': -3768.0,
 'Median': -3768.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': -3768,
                    'x': array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 

In [94]:

SA_instancia3 = knapsack_SA(f3, [g3],n3,p3,w3,c3,initial_solution.GISP_Arr)

In [95]:
args = (initial_solution.solution,1000.0,6.862158749582714e-50)
statistics = get_stats(SA_instancia3, 30, args,showAll=False)

In [96]:
pprint(statistics)

{'Best solution': {'f': -3777,
                   'x': array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0])},
 'Mean': -3769.633333333333,
 'Median': -3768.5,
 'Standard deviation': 2.414999424890661,
 'Worst solution': {'f': -3768,
                    'x': array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
   

## Problema del agente viajero

In [99]:
class TSPFunction:
    def __init__(self, distanceMatrix: (list, np.ndarray)):
        self.distance = np.array(distanceMatrix)
        
    def __call__(self,x: np.ndarray):
        total_dist = self.distance[int(x[-1])][0] + self.distance[0][int(x[0])]
        for i in range(1,len(x)):
            u,v = int(x[i]), int(x[i-1])
            total_dist+= self.distance[u][v]

        return float(total_dist)

class TSPAptitude:
    def __init__(self,  distanceMatrix: (list, np.ndarray)):
        self.f = TSPFunction(distanceMatrix)
        
    def __call__(self,X:np.ndarray):
        return -1 * self.f(X) 

  

In [100]:
def create_travel_matrix(n, low_bound, high_bound):
    matrix = np.zeros((n,n))
    for i in range(1,n):
        for j in range(i):
            value =  np.random.randint(low_bound, high_bound)
            matrix[i][j] = value
            matrix[j][i] = value 
    return matrix

### Instancia 1

In [101]:
matrix1 = create_travel_matrix(10,10,100)
n1 = 10
f1 = TSPFunction(matrix1)

### Instancia 2


In [102]:
matrix2 = create_travel_matrix(15,10,100)
n2 = 15
f2 = TSPFunction(matrix2)

### Instancia 3

In [103]:
matrix3 = create_travel_matrix(20,10,100)
n3 = 20
f3 = TSPFunction(matrix3)

## Búsqueda Tabú

#### Solución inicial

In [104]:
def getInitialSolution(distance_matrix, total_cities):
    Solution = [0]
    remaining_cities  = list(range(1,total_cities))
    
    while len(remaining_cities) != 0:
        from_ =Solution[-1] 
        to_ = remaining_cities[0]
        dist = distance_matrix[from_][to_]
        
        for i in range(1, len(remaining_cities)):
            distance = distance_matrix[from_][remaining_cities[i]]
            if distance < dist:
                to_ = remaining_cities[i]
                dist = distance
        Solution.append(to_)
        ind = remaining_cities.index(to_)
        remaining_cities.pop(ind)
    return Solution

#### Configuración de metaheuristica

In [105]:
class Tabu_Salesman_list:
    def __init__(self,timer):
        self.__TB = {}
        self.timer = timer
    
    def reset(self,timer) -> None:
        self.__TB = {}
        self.timer = timer
        
    def update(self) -> None:
        to_pop = []
        for key in self.__TB:
            if self.__TB[key]-1 == 0:
                to_pop.append(key)
            else:
                self.__TB[key]-=1
        for key in to_pop:
            self.__TB.pop(key)
        
    #x has [p,v,step], we are only interested in v (value)
    def push(self, x : list ) -> None:
        self.__TB[x[1]] = self.timer
        

    def find(self, x : list) -> bool:
        return x[1] in self.__TB
        

In [106]:
class TSP_TS(TS):

    def __init__(self, f_ : function_type , TabuStorage):
        super().__init__(f_,[],TabuStorage)
        

    def get_neighbors(self, x : np.ndarray,**kwargs) -> list: 
        
        neighbors_list = []
        
        ind = random.randint(1,len(x)-1)
        while  self.TL.find([-1,x[ind]]):
            ind = random.randint(1,len(x)-1)
        v = x[ind]
        x_tmp = list(x[v != x])
        for i in range(1, len(x)):
            if ind == i:
                continue
            neighbors_list += [ x_tmp[:i] + [v] + x_tmp[i:]]
            
        return neighbors_list


    def encode_change(self, neighbor : (list,np.ndarray), x : (list,np.ndarray),**kwargs) -> list: #2
        
        x_p ={x[i] : i for i in range(len(x))}
        n_p = {neighbor[i]: i for i in range(len(x))}
        ind = -1
        max_dist = -1
        value = -1
        for i in range(1, len(x)):
            v = x[i]
            dist = abs(x_p[v] - n_p[v])
            if dist > max_dist:
                ind = i
                max_dist = dist
                value = v
       
        return [ind , value]

## Búsqueda Recocido simulado

#### Configuración metaheurística

In [107]:
class TSP_SA(SA):

    def __init__(self, f_ : function_type ):
        super().__init__(f_,[])

    def get_neighbor(self, x : np.ndarray) -> np.ndarray: 
        
        x_ = x.copy()
        N = len(x_)
        index1 = random.randint(1, N-1)
        index2 = random.randint(1, N-1)
    
        while index2 == index1:
            index2 = random.randint(1, N-1)
    
        v = x[index1]
        x_ = list(x_[v != x_])
        x_ = x_[:index2] + [v] + x_[index2:]
        return np.array(x_)


## Búsqueda Algoritmos genéticos

#### Configuración de operadores

In [108]:
configuration_TSP = (GeneticConfig()
                         .cross(crossover.permutation_order_crossover())
                         .mutate(mutation.insertion_mutator(1))
                         .survivor_selection(selection.merge_selector())
                         .parent_selection(selection.tournament_sampler(chunks_=5,prob_=0.8))
                         .fixer_invalide_solutions(NoneFixer()))


#### Configuración de metaheurística

In [109]:
class TSP_GA(GA):
    def __init__(self,  function,\
                        decision_variables:int,\
                        config = None):
        super().__init__(function, decision_variables, [],[], config)
    
    def initialize_population(self, **kwargs) -> np.ndarray:
        individuals = []
        
        for i in range(self.logger['population_size']):
            individuals += [np.random.permutation(range(1,self.Decision_variables))]
            
        return np.array(individuals)

### Resultados

#### Instancia 1

In [113]:
TS_instancia1 = TSP_TS(f1,Tabu_Salesman_list(n1//2))
init_path = np.array(getInitialSolution(matrix1,n1))

In [114]:
args = (init_path, 1140, n1//2, False)
statistics = get_stats(TS_instancia1, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 326.0, 'x': array([0, 5, 1, 9, 4, 3, 2, 7, 8, 6])},
 'Mean': 326.0,
 'Median': 326.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': 326.0, 'x': array([0, 5, 1, 9, 4, 3, 2, 7, 8, 6])}}


In [115]:
SA_instancia1 = TSP_SA(f1)

In [116]:
args = (init_path, 1000.0, 6.862158749582714e-50)
statistics = get_stats(SA_instancia1, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 326.0, 'x': array([0, 5, 1, 9, 4, 3, 2, 7, 8, 6])},
 'Mean': 331.6,
 'Median': 326.0,
 'Standard deviation': 6.8585712797929,
 'Worst solution': {'f': 340.0, 'x': array([0, 5, 1, 9, 4, 3, 2, 7, 6, 8])}}


In [111]:
GA_instancia1 = TSP_GA(TSPAptitude(matrix1),n1,configuration_TSP)

In [143]:
args = (30, 19,0.8, 0.5,False)
statistics = get_stats(GA_instancia1, 30, args, transformer=f1, showAll=False)
pprint(statistics)

{'Best solution': {'f': 326.0,
                   'x': array([5., 1., 9., 4., 3., 2., 7., 8., 6.])},
 'Mean': 361.7,
 'Median': 365.0,
 'Standard deviation': 14.217711958445822,
 'Worst solution': {'f': 389.0,
                    'x': array([4., 9., 1., 5., 8., 7., 6., 3., 2.])}}


#### Instancia 2

In [144]:
TS_instancia2 = TSP_TS(f2,Tabu_Salesman_list(n2//2))
init_path = np.array(getInitialSolution(matrix2,n2))

In [145]:
args = (init_path, 1140, n2//2, False)
statistics = get_stats(TS_instancia2, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 355.0,
                   'x': array([ 0,  2,  1,  5, 14,  6, 10,  3,  7,  9, 13,  4, 12, 11,  8])},
 'Mean': 355.0,
 'Median': 355.0,
 'Standard deviation': 0.0,
 'Worst solution': {'f': 355.0,
                    'x': array([ 0,  2,  1,  5, 14,  6, 10,  3,  7,  9, 13,  4, 12, 11,  8])}}


In [146]:
SA_instancia2 = TSP_SA(f2)

In [147]:
args = (init_path, 1000.0, 6.862158749582714e-50)
statistics = get_stats(SA_instancia2, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 306.0,
                   'x': array([ 0,  2,  1, 12, 11,  6, 10,  3,  7,  4, 13,  9, 14,  5,  8])},
 'Mean': 338.9,
 'Median': 340.5,
 'Standard deviation': 16.493129882873454,
 'Worst solution': {'f': 360.0,
                    'x': array([ 0,  2,  5, 14,  6, 10,  3,  7,  9, 13,  4,  1, 12, 11,  8])}}


In [148]:
GA_instancia2 = TSP_GA(TSPAptitude(matrix2),n2,configuration_TSP)

In [156]:
args = (30, 19,0.8, 0.5,False)
statistics = get_stats(GA_instancia2, 30, args, transformer=f2, showAll=False)
pprint(statistics)

{'Best solution': {'f': 367.0,
                   'x': array([ 8.,  1.,  5., 14.,  9., 13.,  4.,  7.,  6., 10.,  3., 12., 11.,
        2.])},
 'Mean': 425.2,
 'Median': 425.0,
 'Standard deviation': 29.40113376498714,
 'Worst solution': {'f': 481.0,
                    'x': array([ 2.,  3.,  8.,  1., 13.,  9.,  7.,  4., 10.,  5., 12., 11., 14.,
        6.])}}


#### Instancia 3

In [150]:
TS_instancia3 = TSP_TS(f3,Tabu_Salesman_list(n3//2))
init_path = np.array(getInitialSolution(matrix3,n3))

In [151]:
args = (init_path, 1140, n3//2, False)
statistics = get_stats(TS_instancia3, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 446.0,
                   'x': array([ 0,  7,  8,  3, 19, 14,  5, 18, 11,  1, 13, 10,  4, 15,  9, 16, 17,
        2,  6, 12])},
 'Mean': 478.8,
 'Median': 466.5,
 'Standard deviation': 24.32885803594845,
 'Worst solution': {'f': 539.0,
                    'x': array([ 0,  3, 19, 14,  5, 13,  1, 10,  4, 15,  9, 16, 17,  2,  6,  7,  8,
       18, 11, 12])}}


In [152]:
SA_instancia3 = TSP_SA(f3)

In [153]:
args = (init_path, 1000.0, 6.862158749582714e-50)
statistics = get_stats(SA_instancia3, 30, args, showAll=False)
pprint(statistics)

{'Best solution': {'f': 428.0,
                   'x': array([ 0, 12, 15,  4, 10,  2,  6, 14,  5, 13,  1, 18, 11, 19,  3,  8,  7,
        9, 16, 17])},
 'Mean': 503.23333333333335,
 'Median': 504.5,
 'Standard deviation': 33.38431101913326,
 'Worst solution': {'f': 556.0,
                    'x': array([ 0, 18, 16,  4, 10, 15, 12,  7,  8,  3, 19, 14,  6,  2,  5, 13,  1,
       11,  9, 17])}}


In [154]:
GA_instancia3 = TSP_GA(TSPAptitude(matrix3),n3,configuration_TSP)

In [159]:
args = (30, 19,0.8, 0.5,False)
statistics = get_stats(GA_instancia3, 30, args, transformer=f3, showAll=False)
pprint(statistics)

{'Best solution': {'f': 594.0,
                   'x': array([12.,  5., 19.,  2.,  3., 10., 14.,  6., 16.,  9., 15.,  4., 13.,
        1., 11., 18., 17.,  7.,  8.])},
 'Mean': 697.4333333333333,
 'Median': 703.0,
 'Standard deviation': 46.02874705611218,
 'Worst solution': {'f': 778.0,
                    'x': array([17.,  7.,  2.,  8.,  1., 11., 18.,  6., 12., 15.,  9., 16.,  5.,
       19.,  3., 13.,  4., 10., 14.])}}
